In [32]:
from kingdon import Algebra
from sympy import ccode

In [2]:
alg = Algebra(2, 0, 1)

In [6]:
a = alg.multivector(name="a")
b = alg.multivector(name="b")

In [23]:
alg.bin2canon

{0: 'e', 1: 'e0', 2: 'e1', 3: 'e01', 4: 'e2', 5: 'e02', 6: 'e12', 7: 'e012'}

In [30]:
r = a * b

In [34]:
code = ccode(r.values()[0])
code

'a*b + a1*b1 - a12*b12 + a2*b2'

In [36]:
code.replace("a", "a.e").replace("b", "b.e")

'a.e*b.e + a.e1*b.e1 - a.e12*b.e12 + a.e2*b.e2'

In [61]:
# Convert multivector to C++ macro code
def mv_to_code(mv, name):
    lines = []
    for key, value in zip(mv.keys(), mv.values()):
        key = "    r." + alg.bin2canon[key]
        value = ccode(value).replace("a", "a.e").replace("b", "b.e")
        line = key + " += " + value + ";"
        lines.append(line)
    code = "\\\n".join(lines)
    code = "#define {} \\\n".format(name) + code
    return code

fragment = mv_to_code(r, "GP_full")
print(fragment)


#define GP_full \
    r.e += a.e*b.e + a.e1*b.e1 - a.e12*b.e12 + a.e2*b.e2;\
    r.e0 += a.e*b.e0 + a.e0*b.e + a.e01*b.e1 - a.e012*b.e12 + a.e02*b.e2 - a.e1*b.e01 - a.e12*b.e012 - a.e2*b.e02;\
    r.e1 += a.e*b.e1 + a.e1*b.e + a.e12*b.e2 - a.e2*b.e12;\
    r.e2 += a.e*b.e2 + a.e1*b.e12 - a.e12*b.e1 + a.e2*b.e;\
    r.e01 += a.e*b.e01 + a.e0*b.e1 + a.e01*b.e + a.e012*b.e2 - a.e02*b.e12 - a.e1*b.e0 + a.e12*b.e02 + a.e2*b.e012;\
    r.e02 += a.e*b.e02 + a.e0*b.e2 + a.e01*b.e12 - a.e012*b.e1 + a.e02*b.e - a.e1*b.e012 - a.e12*b.e01 - a.e2*b.e0;\
    r.e12 += a.e*b.e12 + a.e1*b.e2 + a.e12*b.e - a.e2*b.e1;\
    r.e012 += a.e*b.e012 + a.e0*b.e12 + a.e01*b.e2 + a.e012*b.e - a.e02*b.e1 - a.e1*b.e02 + a.e12*b.e0 + a.e2*b.e01;


In [69]:
def generate_fragments():
    frags = []

    a = alg.multivector(name="a")
    b = alg.multivector(name="b")
    # Geometric Product in full
    r = a * b
    frag = mv_to_code(r, "GP_FULL")
    frags.append(frag)
    
    # Outer Product in full
    r = a ^ b
    frag = mv_to_code(r, "OP_FULL")
    frags.append(frag)
    
    # Inner Product in full
    r = a | b
    frag = mv_to_code(r, "IP_FULL")
    frags.append(frag)

    # Regressive Product in full
    r = a & b
    frag = mv_to_code(r, "RP_FULL")
    frags.append(frag)

    # TOO EXPENSIVE
    # Sandwitch(Conjugate) Product in full
    # Project in full

    # Geometric Product in grades
    for i in range(4):
        for j in range(4):
            a = alg.multivector(name="a", grades=(i,))
            b = alg.multivector(name="b", grades=(j,))
            r = a * b
            frag = mv_to_code(r, "GP_{}_{}".format(i, j))
            frags.append(frag)

    # Outer Product in grades
    for i in range(4):
        for j in range(4):
            a = alg.multivector(name="a", grades=(i,))
            b = alg.multivector(name="b", grades=(j,))
            r = a ^ b
            frag = mv_to_code(r, "OP_{}_{}".format(i, j))
            frags.append(frag)
    
    # Inner Product in grades
    for i in range(4):
        for j in range(4):
            a = alg.multivector(name="a", grades=(i,))
            b = alg.multivector(name="b", grades=(j,))
            r = a | b
            frag = mv_to_code(r, "IP_{}_{}".format(i, j))
            frags.append(frag)

    # Regressive Product in grades
    # TODO: check is RP splitable
    for i in range(4):
        for j in range(4):
            a = alg.multivector(name="a", grades=(i,))
            b = alg.multivector(name="b", grades=(j,))
            r = a | b
            frag = mv_to_code(r, "RP_{}_{}".format(i, j))
            frags.append(frag)

    return "\n\n".join(frags)

code = generate_fragments()
with open("pga2.inc.h", "w") as f:
    f.write(code)